<a href="https://colab.research.google.com/github/ShreyaSutar2004/Planner_for_farmers/blob/main/Planner_for_farmers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
!pip install gradio

In [46]:
import gradio as gr
import pandas as pd
import requests

In [47]:
df=pd.read_csv('/content/final_crop_planner.csv')
df

,N,P,K,temperature,humidity,ph,rainfall,label,Cropping Season,Harvesting Season,Soil Type
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice,June-September,October-November,"Alluvial Soil, Laterite Soil"
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice,June-September,October-November,"Alluvial Soil, Laterite Soil"
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice,June-September,October-November,"Alluvial Soil, Laterite Soil"
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice,June-September,October-November,"Alluvial Soil, Laterite Soil"
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice,June-September,October-November,"Alluvial Soil, Laterite Soil"
...,...,...,...,...,...,...,...,...,...,...,...
495,113,38,20,22.107190,78.583201,6.364730,74.941366,cotton,June-November,November-January,Black Soil
496,102,53,21,23.038140,76.110215,6.913679,91.496975,cotton,June-November,November-January,Black Soil
497,110,39,18,24.547953,75.397527,7.766260,63.880799,cotton,June-November,November-January,Black Soil
498,107,58,15,23.738680,75.775038,7.556064,76.636692,cotton,June-November,November-January,Black Soil


In [48]:
crop_ranges = df.groupby("label").agg({
    "temperature": ["min", "max"],
    "humidity": ["min", "max"]
}).reset_index()
crop_ranges.columns = ["Crop", "Temp_Min", "Temp_Max", "Humidity_Min", "Humidity_Max"]

In [49]:
def get_weather(location):
    API_KEY = "UYKEUTWELDHK9WTNQTFJC8HLJ"
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Pune?unitGroup=us&key=UYKEUTWELDHK9WTNQTFJC8HLJ&contentType=json"
    response = requests.get(url).json()
    temp = (response["days"][0]["temp"] - 32) * 5.0/9.0  # Convert Fahrenheit to Celsius
    humidity = response["days"][0]["humidity"]
    return temp, humidity


In [50]:
def recommend(location, crop, soil_type, season):
    temp, humidity = get_weather(location)

    # Fetch crop-specific optimal ranges
    crop_data = crop_ranges[crop_ranges["Crop"] == crop]
    if crop_data.empty:
        return "Crop not found in dataset."

    temp_min, temp_max = crop_data.iloc[0][["Temp_Min", "Temp_Max"]]
    humidity_min, humidity_max = crop_data.iloc[0][["Humidity_Min", "Humidity_Max"]]

    # Compare inputs with ideal conditions
    if temp < temp_min or temp > temp_max or humidity < humidity_min or humidity > humidity_max:
        return f"Soil preparation and Fertilizing within the next few days!\nCurrent Temperature: {temp:.2f}°C\nCurrent Humidity: {humidity}%"

    if season in df[df["label"] == crop]["Cropping Season"].values:
       return f"Time for Sowing and Irrigation\nCurrent Temperature: {temp:.2f}°C\nCurrent Humidity: {humidity}%"

    if season in df[df["label"] == crop]["Harvesting Season"].values:
       return f"Pesticide management, Harvesting and Storing\nCurrent Temperature: {temp:.2f}°C\nCurrent Humidity: {humidity}%"

       return f"General Maintenance Required\nCurrent Temperature: {temp:.2f}°C\nCurrent Humidity: {humidity}%"

In [51]:
iface = gr.Interface(
    fn=recommend,
    inputs=[
        gr.Textbox(label="Location"),
        gr.Dropdown(choices=df["label"].unique().tolist(), label="Crop"),
        gr.Textbox(label="Soil Type"),
        gr.Textbox(label="Present Month")
    ],
    outputs=gr.Textbox(label="Recommendation"),
    title="Farmer's Planner",
    description="Enter the location and crop details to get recommendations for optimal farming activities."
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://237966f20ff7051419.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
